<h1>Invoke SageMaker Prediction Service</h1>

In [ ]:
## Standard Python Libraries
%matplotlib inline
import sys
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import math
import os

## SageMaker Libraries
import boto3
import re
from sagemaker import get_execution_role
import sagemaker

In [ ]:
# Acquire a realtime endpoint
#endpoint_name = 'xgboost-biketrain-v1'
endpoint_name = 'sagemaker-xgboost-2020-07-06-12-05-58-891'
predictor = sagemaker.predictor.RealTimePredictor(endpoint=endpoint_name)

In [ ]:
from sagemaker.predictor import csv_serializer, json_deserializer

predictor.content_type = 'text/csv'
predictor.serializer = csv_serializer
predictor.deserializer = None

In [ ]:
df_all = pd.read_csv('bike_test.csv',header=None)

In [ ]:
df_all.head()

In [ ]:
df_all.columns[1:]

In [ ]:
# Need to pass an array to the prediction
# can pass a numpy array or a list of values [[19,1],[20,1]]
arr_test = df_all.as_matrix(df_all.columns[1:])

In [ ]:
type(arr_test)

In [ ]:
arr_test.shape

In [ ]:
arr_test[:5]

In [ ]:
result = predictor.predict(arr_test[:2])

In [ ]:
# decode method has to be used
result

In [ ]:
arr_test.shape

In [ ]:
# For large number of predictions, we can split the input data and
# Query the prediction service.
# array_split is convenient to specify how many splits are needed
predictions = []
for arr in np.array_split(arr_test,10):
    result = predictor.predict(arr)
    result = result.decode("utf-8")
    result = result.split(',')
    print (arr.shape)
    predictions += [float(r) for r in result]

In [ ]:
len(predictions)

In [ ]:
np.expm1(predictions)

In [ ]:
df_all['count'] = np.expm1(predictions)

In [ ]:
df_all.head()

In [ ]:
df_all = df_all.rename(columns={0: 'datetime'})
df_all[['datetime','count']].to_csv('predicted_count_cloud.csv',index=False)